# Scence

As all of us now, in many sitcomes and seinfled spesificly the show is a series of scene.  
Many times, two seperate scence are completly different (for example Gorege and Jerry are in the street, next scene they sit togather in a resturant).  
Since we excpt following sentances will help us better predict, we shouldn't consider different scence as following sentacnes.  
Moreover, we remmber kramer break in to the room and the audince laughing.  

After watching enough senfiled eppsiodes we now that sometimes we have this _[transiation music](https://www.youtube.com/watch?v=EP2GMyQdw78)_ it's not always the case, and we did see different timings of this interlude.

In [3]:
from utils import load_corpus, split_train_test
# from train_utils import split_train_test
from compare_models import plot_confusion_matrix, compare_models_roc_curve
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
%load_ext autoreload
%autoreload 2

df = load_corpus()

Loading BokehJS ...

Let's start with finiding the time between following sentances, we shall remmber that new episodes should have time zero

In [4]:
df.loc[:, 'time_from_prev'] = np.array([0] + [df.start[i] - df.end[i - 1]
                                       if df.episode_num[i] == df.episode_num[i - 1] else 0
                                       for i in range(1, len(df.start))])


In [5]:
p = figure(plot_height=300, tools="pan,box_zoom,reset, save")

hist, edges = np.histogram(df.time_from_prev, bins=np.arange(-0.1,50,0.2))
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], alpha=0.4)

show(p)

What do we see?  
As excpected, most of the following sentances occur after < 1sec.  
But we are intrested in the long tail, it's very weird to see distance of > 40sec, let's look at them

In [6]:
df.iloc[[j for i in df[(df.time_from_prev > 40)].index for j in [i-1, i]]].head(6)

,avg_word_length,character,end,episode_name,episode_num,is_funny,laugh_time,length,line_num,num_words,season,start,total_lines,txt,global_episode_num,time_from_prev
7147,4.000000,ELAINE,1255.228,The Smelly Car,21,True,1255.3,1.517,461,2.0,4,1253.711,475,Sauce me.,15,0.000
7148,2.800000,ELAINE,1302.299,The Smelly Car,21,False,NaN,2.125,462,5.0,4,1300.174,475,Why do we need BO?,15,44.946
11216,5.400000,GEORGE,426.385,The Barber,8,True,451.5,2.694,176,5.0,5,423.691,445,"Well, we'll straighten him out.",24,0.074
11217,3.285714,GEORGE,472.437,The Barber,8,False,NaN,1.533,177,7.0,5,470.904,445,What do you do there all day?,24,44.519
21258,4.000000,STEIN,1283.827,The Jimmy,19,True,1284.7,5.299,441,11.0,6,1278.528,454,I wanna dedicate this song to a very courageo...,44,0.741
21259,4.600000,KRAMER,1390.368,The Jimmy,19,False,NaN,2.398,442,5.0,6,1387.970,454,"Hey, got the new Penthouse.",44,104.143


After watching some episodes that's look allright.  
It's really a new scene, and it there is that match seconds without speaking.  
  
  
We prefer to have false negative than false positive --> we prefer to have two scences togather than to break in a middle of a scene, so we will pick the threshold a bit high, let's say 5 seconds

In [7]:
df.episode_num.sample(1).values[0]

3

In [8]:
# df.loc[:, ['new_scene']] = df['time_from_prev'] > 5
# # Other than that, if it's a new episode we want new scene, used heuristic for that
# df.loc[(df.time_from_prev == 0) & (df.start <= 2), ['new_scene']] = True

df[(df.episode_num == df.episode_num.sample(1).values[0]) & (df.season == df.season.sample(1).values[0])]

,avg_word_length,character,end,episode_name,episode_num,is_funny,laugh_time,length,line_num,num_words,season,start,total_lines,txt,global_episode_num,time_from_prev
42085,3.777778,GEORGE,6.053,The Dealership,11,False,NaN,2.076,0,8.0,9,3.977,469,When are they gonna have the flying cars?,88,0.000
42086,4.428571,JERRY,7.807,The Dealership,11,False,NaN,1.578,1,6.0,9,6.229,469,"Yeah, they have been promising that.",88,0.176
42087,3.785714,GEORGE,11.148,The Dealership,11,True,9.1,3.167,2,13.0,9,7.981,469,Years. When we were kids it seemed like it wa...,88,0.174
42088,3.428571,JERRY,13.476,The Dealership,11,True,13.4,2.158,3,7.0,9,11.318,469,I think Ed Begley Jr. Has one.,88,0.170
42089,5.500000,GEORGE,15.313,The Dealership,11,False,NaN,1.660,4,4.0,9,13.653,469,"No, that's just electric.",88,0.177
42090,4.000000,JERRY,18.146,The Dealership,11,False,NaN,2.491,5,10.0,9,15.655,469,What about Harrison Ford? He had one in Blade...,88,0.342
42091,3.200000,JERRY,19.736,The Dealership,11,False,NaN,1.411,6,5.0,9,18.325,469,That was a cool one.,88,0.179
42092,5.222222,GEORGE,22.579,The Dealership,11,True,22.7,2.669,7,8.0,9,19.910,469,"Well, what's the competition? Chitty Chitty B...",88,0.174
42093,3.222222,JERRY,24.454,The Dealership,11,False,NaN,1.708,8,8.0,9,22.746,469,What do you think the big holdup is?,88,0.167
42094,3.833333,GEORGE,27.916,The Dealership,11,False,NaN,3.085,9,11.0,9,24.831,469,The government is very touchy about us being ...,88,0.377


Not fantastic, we still see FP